In [ ]:
#!conda info
#!conda list

In [ ]:
import requests
import json
import sqlalchemy
from IPython.display import JSON
from sqlalchemy import Table, Column, Integer, Float, String, MetaData, DateTime, create_engine
from sqlalchemy.orm import sessionmaker
import datetime
import mysql.connector
import config #make sure this matches the name of your config file
import time

In [ ]:
# Request Data from API
engine = create_engine(f"mysql+mysqlconnector://{config.user}:{config.passw}@{config.uri}:3306/wheelieGood", echo=True)

requests_stations = requests.get(config.STATIONS, params={"apiKey": config.APIKEY, "contract": config.NAME})
weather_request = requests.get(config.WEATHER_INFO, params={"appid": config.WAPIKEY, "id": config.WNAME})

# x = JSON(json.loads(requests_stations.text))

# json = json.dumps(x)
# f = open("test.json", "w")
# f.write(json)
# f.close()


In [ ]:
# Create weather & bike table & columns
meta = MetaData()

# Creating bike table and columns

stations = Table(
'stations', meta,
Column('number', Integer),
Column('name', String(128)),
Column('address', String(128)),
Column('pos_lat', Float),
Column('pos_lng', Float),
Column('bike_stands', Integer),
Column('available_bike_stands', Integer),
Column('available_bikes', Integer),
Column('last_update', DateTime)
)

# Creating weather table and columns

weather = Table(
'weather', meta,
Column('weather_main', String(128)),
Column('weather_description', String(128)),
Column('weather_icon', String(128)),
Column('main_temp', Float),
Column('main_pressure', Integer),
Column('main_humidity', Integer),
Column('main_temp_min', Float),
Column('main_temp_max', Float),
Column('visibility', Integer),
Column('wind_speed', Float),
Column('wind_deg', Integer),
Column('clouds_all', Integer),
Column('dt', DateTime),
Column('sys_type', Integer),
Column('sys_id', Integer),
Column('sys_country', String(128)),
Column('sys_sunrise', DateTime),
Column('sys_sunset', DateTime),
Column('city_id', Integer),
Column('city_name', String(128)),
Column('cod', Integer)
)

meta.create_all(engine)

In [ ]:
# Getting bike data

def get_station(obj):
    return {'number': obj['number'],
            'name': obj['name'],
            'address': obj['address'],
            'pos_lng': obj['position']['lng'],
            'pos_lat': obj['position']['lat'],
            'bike_stands': obj['bike_stands'],
            'available_bike_stands': obj['available_bike_stands'],
            'available_bikes': obj['available_bikes'],
            'last_update': datetime.datetime.fromtimestamp(int(obj['last_update'] / 1e3))
           }


In [ ]:
# Getting weather data

def get_weather(weather):
    
    weather = weather_request.json()

    weather['weather_id'] = weather['weather'][0]['id']
    weather['weather_main'] = weather['weather'][0]['main']
    weather['weather_description'] = weather['weather'][0]['description']
    weather['weather_icon'] = weather['weather'][0]['icon']
    weather['main_temp'] = weather['main']['temp']
    weather['main_pressure'] = weather['main']['pressure']
    weather['main_humidity'] = weather['main']['humidity']
    weather['main_temp_min'] = weather['main']['temp_min']
    weather['main_temp_max'] = weather['main']['temp_max']
    weather['visibility'] = weather['visibility']
    weather['wind_speed'] = weather['wind']['speed']
    weather['wind_deg'] = weather['wind']['deg']
    weather['clouds_all'] = weather['clouds']['all']
    weather['dt'] = datetime.datetime.fromtimestamp(int(weather['dt']))
    weather['sys_type'] = weather['sys']['type']
    weather['sys_id'] = weather['sys']['id']
    weather['sys_country'] = (weather['sys']['country'])
    weather['sys_sunrise'] = datetime.datetime.fromtimestamp(int(weather['sys']['sunrise']))
    weather['sys_sunset'] = datetime.datetime.fromtimestamp(int(weather['sys']['sunset']))
    weather['city_id'] = weather['id']
    weather['city_name'] = weather['name']
    weather['cod'] = weather['cod']
        
    return weather


In [ ]:
# Execute every 5 minutes

while True:
    try:
        weather_insert = list(map(get_weather, weather_request.json()))
        print(weather_insert)
        engine.execute(weather.insert(),weather_insert)
        
        values = list(map(get_station, requests_stations.json()))
        print(values)
        ins = stations.insert().values(values)
        engine.execute(ins)
        
        time.sleep(5 * 60)
    except:
        print("Please try again")